In [1]:
base_url = "https://energy.cngold.org//list_427_all.html"
from urllib import request

from bs4 import BeautifulSoup as bs
import json
import re


class get_Oil_price:

    def __init__(self, base_url):
        self.base_Url = base_url

    #获取日期URL
    def get_list_url(self):
        html = request.urlopen(self.base_Url)
        soup = bs(html, "html5lib")
        yiqi_li = soup.find("div",text = "2018年柴油资讯列表").parent
        date_url_list = []
        if yiqi_li:
            date_list = yiqi_li.findAll("li")
            for i in date_list:
                a_tag = i.find("a")
                date_url_list.append((a_tag.text, a_tag.attrs["href"]))
            return date_url_list
        else:
            print("没有获取种子页")
            raise BaseException

    def get_list_page(self, date_url_tuple):
        date, date_url = date_url_tuple[0], date_url_tuple[1]
        html = request.urlopen(date_url)
        soup = bs(html)
        detail_li = soup.find("a", text = re.compile("\w+最新\w*柴油\w*价格\w+"))
        if detail_li:
            detail_url = detail_li.attrs["href"]
            return (date, detail_url)
        else:
            pass

    def parse_detail_page(self, detail_url_tuple):
        date, detail_url = detail_url_tuple[0], detail_url_tuple[1]
        html = request.urlopen(detail_url)
        soup = bs(html)
        table = soup.find("table")
        tr_list = table.findAll("tr")
        district_list = []
        unit_price_list = []
        for i, element in enumerate(tr_list):
            if i==0:
                td_list = element.findAll("td")
                for j, tag in enumerate(td_list):
                    if j==0:
                        district = tag.text.strip()
                    elif j==1:
                        unit_price = tag.text.strip()
            else:
                td_list = element.findAll("td")
                for j, tag in enumerate(td_list):
                    if j==0:
                        district_list.append(tag.text.strip())
                    elif j==1:
                        unit_price_list.append(tag.text.strip())
        return (date,{district:district_list,unit_price:unit_price_list})

    def get_oil_info(self):
        date_url_list = self.get_list_url()
        res_dic = {}
        for i in date_url_list:
            detail_dict = self.get_list_page(i)
            if detail_dict:
                date, middle_res = self.parse_detail_page(detail_dict)
                res_dic[date] = middle_res
        return res_dic


In [ ]:
obj = get_Oil_price("https://energy.cngold.org//list_427_all.html")
res_js = obj.get_oil_info()


C:\Anaconda2\envs\idp\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Anaconda2\envs\idp\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [15]:
date_info = []
for i in res_js.keys():
    day_info = res_js[i]
    try:
        columns = day_info["地区"]
    except:
        columns = None
    if columns:
        day_df = pd.DataFrame(day_info["今日柴油(元/升)"], index = day_info["地区"],
                              columns = [i])
        date_info.append(day_df)

In [16]:
res = pd.concat(date_info,axis = 1)

In [18]:
writer = pd.ExcelWriter("oil_day_price.xlsx")
res.to_excel(writer, "sheet1")
writer.save()

In [19]:
res_js

{'2017-01-03': {'今日柴油(元/升)': ['6.13',
   '6.07',
   '6.09',
   '6.03',
   '5.98',
   '6.09',
   '6',
   '6.1',
   '6.14',
   '5.48',
   '6.17',
   '6.09',
   '6.07',
   '5.32',
   '5.99',
   '6.15',
   '5.86',
   '6.06',
   '6.13',
   '5.29',
   '6.89',
   '5.99',
   '6.03',
   '6.08',
   '6.15',
   '6',
   '6.05',
   '6.46',
   '5.52',
   '6.03',
   '6.08'],
  '地区': ['北京',
   '上海',
   '天津',
   '重庆',
   '安徽',
   '福建',
   '甘肃',
   '广东',
   '广西',
   '贵州',
   '海南',
   '河北',
   '河南',
   '黑龙江',
   '湖北',
   '湖南',
   '吉林',
   '江苏',
   '江西',
   '辽宁',
   '内蒙古',
   '宁夏',
   '青海',
   '山东',
   '山西',
   '陕西',
   '四川',
   '西藏',
   '新疆',
   '云南',
   '浙江']},
 '2017-01-04': {'今日柴油(元/升)': ['6.13',
   '6.07',
   '6.09',
   '6.03',
   '5.98',
   '6.09',
   '6',
   '6.1',
   '6.14',
   '5.48',
   '6.17',
   '6.09',
   '6.07',
   '5.32',
   '5.99',
   '6.15',
   '5.86',
   '6.06',
   '6.13',
   '5.29',
   '6.89',
   '5.99',
   '6.03',
   '6.08',
   '6.15',
   '6',
   '6.05',
   '6.46',
   '5.52',
   '6.03',

In [1]:
import pandas as pd

In [2]:
detail_weather_loc = pd.read_csv("detail_weather_loc_bac.csv")

In [3]:
oil_price_df = pd.read_excel("oil_day_price.xlsx")

In [4]:
detail_weather_loc.head()

,Unnamed: 0,日期,一级品类,详细品类,订单类型,交易类型,货值（吨/元）,起运地经度,起运地纬度,目的地经度,...,start_wind_scale,district_y,city_y,province_y,desti_url,desti_max_temp,desti_min_temp,desti_weather,desti_wind_direct,desti_wind_scale
0,0,2017-01-01,1,148,0,1,3333.33,114.203697,36.696506,114.670032,...,1级,成安县,邯郸市,河北省,http://lishi.tianqi.com/chengan/201701.html,3.0,-2.0,雾,北风,1级
1,1,2017-01-01,1,148,0,1,3294.89,117.738658,39.900401,114.309091,...,1级,江岸区,武汉市,湖北省,http://lishi.tianqi.com/jianganqu/201701.html,13.0,4.0,多云,北风,微风
2,2,2017-01-01,1,135,0,1,2857.14,117.200983,39.084158,108.940174,...,2级,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2017-01-01,2,225,0,1,8064.52,113.401679,35.099378,117.063754,...,1级,宜秀区,安庆市,安徽省,http://lishi.tianqi.com/yixiuqu/201701.html,15.0,4.0,多云,东南风,1级
4,4,2017-01-01,1,135,0,1,3672.87,118.402893,35.089917,118.909246,...,1级,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
oil_price_df = oil_price_df.reset_index()

In [6]:
type(oil_price_df["2017-12-29"])

pandas.core.series.Series

In [7]:
len_series = len(oil_price_df)
res = []
for i in oil_price_df.columns[1:]:
    sub_series = oil_price_df.loc[:,["index",i]]
    sub_series = sub_series.rename(columns = {"index":"省直辖市", i:"oil_unit_price"})
    
    date_df = pd.DataFrame(len_series*[i], columns = ["日期"])
    
    sub_res = pd.concat([sub_series, date_df], axis = 1)
    res.append(sub_res)    

In [8]:
oil_price = pd.concat(res, axis = 0, ignore_index = True)

In [30]:
oil_price_df.to_csv("oil_price_df.csv", encoding = "utf-8")

In [118]:
oil_price_df.head()

,index,2017-12-29,2017-12-28,2017-12-27,2017-12-26,2017-12-25,2017-12-22,2017-12-21,2017-12-20,2017-12-19,...,2017-01-17,2017-01-16,2017-01-13,2017-01-12,2017-01-11,2017-01-10,2017-01-09,2017-01-05,2017-01-04,2017-01-03
0,上海,6.37,6.37,6.37,6.37,6.37,6.37,6.37,6.37,6.37,...,5.99,6.07,6.07,6.07,6.07,6.07,6.07,6.07,6.07,6.07
1,云南,6.47,6.47,6.47,6.47,6.47,6.47,6.47,6.47,6.47,...,5.95,6.03,6.03,6.03,6.03,6.03,6.03,6.03,6.03,6.03
2,内蒙古,5.87,5.87,5.87,5.87,5.87,5.87,5.87,5.87,5.87,...,6.89,6.89,6.89,6.89,6.89,6.89,6.89,6.89,6.89,6.89
3,北京,6.43,6.43,6.43,6.43,6.43,6.43,6.43,6.43,6.43,...,6.05,6.13,6.13,6.13,6.13,6.13,6.13,6.13,6.13,6.13
4,吉林,6.31,6.31,6.31,6.31,6.31,6.31,6.31,6.31,6.31,...,5.78,5.86,5.86,5.86,5.86,5.86,5.86,5.86,5.86,5.86


In [35]:
oil_price.to_csv("oil_price.csv", encoding = "utf-8")

In [119]:
oil_price

,省直辖市,oil_unit_price,日期
0,上海市,6.37,2017-12-29
1,云南省,6.47,2017-12-29
2,内蒙古自治区,5.87,2017-12-29
3,北京市,6.43,2017-12-29
4,吉林省,6.31,2017-12-29
5,四川省,6.49,2017-12-29
6,天津市,6.39,2017-12-29
7,宁夏回族自治区,6.29,2017-12-29
8,安徽省,6.42,2017-12-29
9,山东省,6.38,2017-12-29


In [13]:
from urllib import request
from bs4 import BeautifulSoup as bs

In [14]:
html = request.urlopen("https://zhidao.baidu.com/question/1688966905471090548.html")
soup = bs(html)

C:\Anaconda2\envs\idp\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Anaconda2\envs\idp\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [46]:
text = "<br/>北京市 京 北京 <br/>浙江省 浙 杭州 <br/>天津市 津 天津 <br/>安徽省 皖 合肥 <br/>上海市 沪 上海 <br/>福建省 闽 福州 <br/>重庆市 渝 重庆 <br/>江西省 赣 南昌 <br/>山东省 鲁 济南  <br/>河南省 豫 郑州 <br/>内蒙古自治区 内蒙古 呼和浩特 <br/>湖北省 鄂 武汉 <br/>新疆维吾尔自治区 新 乌鲁木齐 <br/>湖南省 湘 长沙 <br/>宁夏回族自治区 宁 银川 <br/>广东省 粤 广州 <br/>西藏自治区 藏 拉萨 <br/>海南省 海口 <br/>广西壮族自治区 桂 南宁 <br/>四川省 川或蜀 成都 <br/>河北省 冀 石家庄 <br/>贵州省 贵或黔 贵阳 <br/>山西省 晋 太原 <br/>云南省 云或滇 昆明 <br/>辽宁省 辽 沈阳 <br/>陕西省 陕或秦 西安 <br/>吉林省 吉 长春 <br/>甘肃省 甘或陇 兰州 <br/>黑龙江省 黑 哈尔滨 <br/>青海省 青 西宁 <br/>江苏省 苏 南京<br/>"

In [47]:
text_list = text.split("<br/>")

In [48]:
res = []
for i in text_list:
    if i != "":
        sub = i.split()[0]
        res.append(sub)

In [49]:

test = oil_price["省直辖市"].drop_duplicates().values
print(len(res), len(test))

31 31


In [36]:
oil_price.head()

,省直辖市,oil_unit_price,日期
0,上海,6.37,2017-12-29
1,云南,6.47,2017-12-29
2,内蒙古,5.87,2017-12-29
3,北京,6.43,2017-12-29
4,吉林,6.31,2017-12-29


In [51]:
test

array(['上海', '云南', '内蒙古', '北京', '吉林', '四川', '天津', '宁夏', '安徽', '山东', '山西',
       '广东', '广西', '新疆', '江苏', '江西', '河北', '河南', '浙江', '海南', '湖北', '湖南',
       '甘肃', '福建', '西藏', '贵州', '辽宁', '重庆', '陕西', '青海', '黑龙江'],
      dtype=object)

In [53]:
res = sorted(res)

In [54]:
province_dic = dict(zip(test,res))

In [55]:
province_dic

{'上海': '上海市',
 '云南': '云南省',
 '内蒙古': '内蒙古自治区',
 '北京': '北京市',
 '吉林': '吉林省',
 '四川': '四川省',
 '天津': '天津市',
 '宁夏': '宁夏回族自治区',
 '安徽': '安徽省',
 '山东': '山东省',
 '山西': '山西省',
 '广东': '广东省',
 '广西': '广西壮族自治区',
 '新疆': '新疆维吾尔自治区',
 '江苏': '江苏省',
 '江西': '江西省',
 '河北': '河北省',
 '河南': '河南省',
 '浙江': '浙江省',
 '海南': '海南省',
 '湖北': '湖北省',
 '湖南': '湖南省',
 '甘肃': '甘肃省',
 '福建': '福建省',
 '西藏': '西藏自治区',
 '贵州': '贵州省',
 '辽宁': '辽宁省',
 '重庆': '重庆市',
 '陕西': '陕西省',
 '青海': '青海省',
 '黑龙江': '黑龙江省'}

In [57]:
def trans_province(x, province_dic = province_dic):
    return province_dic[x]

In [58]:
oil_price["省直辖市"] = oil_price["省直辖市"].map(trans_province)

In [59]:
oil_price

,省直辖市,oil_unit_price,日期
0,上海市,6.37,2017-12-29
1,云南省,6.47,2017-12-29
2,内蒙古自治区,5.87,2017-12-29
3,北京市,6.43,2017-12-29
4,吉林省,6.31,2017-12-29
5,四川省,6.49,2017-12-29
6,天津市,6.39,2017-12-29
7,宁夏回族自治区,6.29,2017-12-29
8,安徽省,6.42,2017-12-29
9,山东省,6.38,2017-12-29


In [80]:
import time

In [83]:
date_range  = pd.date_range(start = "2017-01-01", end = "2017-12-31")
date_range = pd.DataFrame(date_range.format(), columns = ["日期"])

In [68]:
len(date_range)

365

In [92]:
transed_res = []
for i in res:
    sub = oil_price[oil_price["省直辖市"]==i].merge(date_range, on = ["日期"],
                                                how = "right").sort_values(by = "日期")
    sub = sub.fillna(method = "ffill")
    sub = sub.fillna(method = "bfill")
    transed_res.append(sub)

In [93]:
transed_res = pd.concat(transed_res)

In [95]:
transed_res[transed_res["oil_unit_price"].isnull()]

,省直辖市,oil_unit_price,日期


In [104]:
transed_res = transed_res.rename(columns = {"省直辖市":"start_province"})

In [105]:
transed_res.head()

,start_province,oil_unit_price,日期
251,上海市,6.07,2017-01-01
252,上海市,6.07,2017-01-02
250,上海市,6.07,2017-01-03
249,上海市,6.07,2017-01-04
248,上海市,6.07,2017-01-05


In [106]:
total_info = detail_weather_loc.merge(transed_res, on = ["日期", "start_province"],how = "left")

In [108]:
transed_res = transed_res.rename(columns = {"start_province":"desti_province"})

In [109]:
total_info = total_info.merge(transed_res, on = ["日期", "desti_province"], how = "left")

In [110]:
len(total_info)

213645

In [111]:
total_info.head()

,Unnamed: 0,日期,一级品类,详细品类,订单类型,交易类型,货值（吨/元）,起运地经度,起运地纬度,目的地经度,...,city_y,province_y,desti_url,desti_max_temp,desti_min_temp,desti_weather,desti_wind_direct,desti_wind_scale,oil_unit_price_x,oil_unit_price_y
0,0,2017-01-01,1,148,0,1,3333.33,114.203697,36.696506,114.670032,...,邯郸市,河北省,http://lishi.tianqi.com/chengan/201701.html,3.0,-2.0,雾,北风,1级,6.09,6.09
1,1,2017-01-01,1,148,0,1,3294.89,117.738658,39.900401,114.309091,...,武汉市,湖北省,http://lishi.tianqi.com/jianganqu/201701.html,13.0,4.0,多云,北风,微风,6.09,5.99
2,2,2017-01-01,1,135,0,1,2857.14,117.200983,39.084158,108.940174,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.09,6.00
3,3,2017-01-01,2,225,0,1,8064.52,113.401679,35.099378,117.063754,...,安庆市,安徽省,http://lishi.tianqi.com/yixiuqu/201701.html,15.0,4.0,多云,东南风,1级,6.07,5.98
4,4,2017-01-01,1,135,0,1,3672.87,118.402893,35.089917,118.909246,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.08,6.06


In [113]:
total_info.drop(["Unnamed: 0"], axis = 1, inplace = True)

In [114]:
total_info.head()

,日期,一级品类,详细品类,订单类型,交易类型,货值（吨/元）,起运地经度,起运地纬度,目的地经度,目的地纬度,...,city_y,province_y,desti_url,desti_max_temp,desti_min_temp,desti_weather,desti_wind_direct,desti_wind_scale,oil_unit_price_x,oil_unit_price_y
0,2017-01-01,1,148,0,1,3333.33,114.203697,36.696506,114.670032,36.444317,...,邯郸市,河北省,http://lishi.tianqi.com/chengan/201701.html,3.0,-2.0,雾,北风,1级,6.09,6.09
1,2017-01-01,1,148,0,1,3294.89,117.738658,39.900401,114.309091,30.600064,...,武汉市,湖北省,http://lishi.tianqi.com/jianganqu/201701.html,13.0,4.0,多云,北风,微风,6.09,5.99
2,2017-01-01,1,135,0,1,2857.14,117.200983,39.084158,108.940174,34.341568,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.09,6.00
3,2017-01-01,2,225,0,1,8064.52,113.401679,35.099378,117.063754,30.543494,...,安庆市,安徽省,http://lishi.tianqi.com/yixiuqu/201701.html,15.0,4.0,多云,东南风,1级,6.07,5.98
4,2017-01-01,1,135,0,1,3672.87,118.402893,35.089917,118.909246,32.096228,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.08,6.06


In [115]:
total_info.columns

Index(['日期', '一级品类', '详细品类', '订单类型', '交易类型', '货值（吨/元）', '起运地经度', '起运地纬度',
       '目的地经度', '目的地纬度', '运距', '车型', '车长', '运价（元/吨）',
       'start_formatted_address', 'start_district', 'start_city',
       'start_province', 'start_township', 'desti_formatted_address',
       'desti_district', 'desti_city', 'desti_province', 'desti_township',
       'district_x', 'city_x', 'province_x', 'start_url', 'start_max_temp',
       'start_min_temp', 'start_weather', 'start_wind_direct',
       'start_wind_scale', 'district_y', 'city_y', 'province_y', 'desti_url',
       'desti_max_temp', 'desti_min_temp', 'desti_weather',
       'desti_wind_direct', 'desti_wind_scale', 'oil_unit_price_x',
       'oil_unit_price_y'],
      dtype='object')

In [116]:
total_info.to_csv("total_info.csv", encoding = "utf-8")